In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
trainset = datasets.MNIST('/Users/rames/Documents/IITB/Others/SOS 2022', download=True, train=True, transform=transform)
valset = datasets.MNIST('/Users/rames/Documents/IITB/Others/SOS 2022', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw\train-images-idx3-ubyte.gz to /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw\train-labels-idx1-ubyte.gz to /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw\t10k-images-idx3-ubyte.gz to /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw\t10k-labels-idx1-ubyte.gz to /Users/rames/Documents/IITB/Others/SOS 2022\MNIST\raw



In [3]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [4]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

In [5]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)


In [6]:
logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.6220128871738784
Epoch 1 - Training loss: 0.2722388850505164
Epoch 2 - Training loss: 0.21060611475616503
Epoch 3 - Training loss: 0.1706704252393547
Epoch 4 - Training loss: 0.14266286232173125
Epoch 5 - Training loss: 0.12271888093597917
Epoch 6 - Training loss: 0.10911696298476825
Epoch 7 - Training loss: 0.09535033581679118
Epoch 8 - Training loss: 0.08581326048531289
Epoch 9 - Training loss: 0.07826940644321952
Epoch 10 - Training loss: 0.07076279554309557
Epoch 11 - Training loss: 0.06460580904024846
Epoch 12 - Training loss: 0.06039479019707129
Epoch 13 - Training loss: 0.054313938681985426
Epoch 14 - Training loss: 0.051546007531063594

Training Time (in minutes) = 5.926488637924194


In [8]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9726


In [9]:
torch.save(model, './my_mnist_model.pt')